In [1]:
import numpy as np


## Linear Equations
$$
Ax = b
$$
### Solution 1:
- Fixed point iteration:

    define $G(x) = Ax - b + x = ( A + I)x - b$, and let $G(x)$ be the x in the next iteration
    
    The iterative process converges if all eigenvalues of (A+I) is within the unit circle. However, this condition is too restrict, which is unlikely to happen in real cases. Therefore, we use Gauss-Jacobi, Gauss-Seidel and other iterative shemes instead.
    
    

### Gauss-Jacobi algorithm

In [5]:
def gauss_jacobi(A,b, x):
    # Extract the diagonal entries
    N = np.diagflat(np.diag(A))
    P = N - A
    return np.linalg.solve(N, b + P.dot(x))

In [6]:

A = np.array([[3,1,2,-2],[1,5,2,-3],[2,3,6,3],[1,1,2,-5]])
x = np.zeros(shape = len(A))
b = np.random.uniform(high = 5,size=len(A))
epsilon = 1e-14
pre_x = np.zeros_like(x)
while True:
    pre_x = x
    x = gauss_jacobi(A,b,x)
    if  ((x - pre_x)**2).sum() < epsilon:
        print('Successfully find solutions:', x)
        break

Successfully find solutions: [-1.01708116  0.16642876  0.93157338 -0.72527202]


### Gauss-Seidel algorithm

In [7]:
def gauss_seidel(A,b, x,w = 1,shuffle = False):
        # Allowing dampening and extrapolating with weight w
        # The sequence of solving x's matters for Gauss-Seidel algorithm, we add a shuffle option
    if shuffle == True:
        values = np.concatenate([A,b.reshape(len(b),-1)],axis=1)
        A = values[:,:-1]
        b = values[:,-1]
    
    # Extract the diagonal entries, lower triangular, and upper triangular
    L = np.tril(A)
    U = A - L
        
    return w * np.linalg.solve(L, b - U.dot(x)) + (1 - w) * x

In [8]:
A

array([[ 3,  1,  2, -2],
       [ 1,  5,  2, -3],
       [ 2,  3,  6,  3],
       [ 1,  1,  2, -5]])

In [9]:
N = np.diag(A)

In [10]:
(np.diag(A)).shape

(4,)

In [11]:
A - np.diagflat(np.diag(A))

array([[ 0,  1,  2, -2],
       [ 1,  0,  2, -3],
       [ 2,  3,  0,  3],
       [ 1,  1,  2,  0]])

In [12]:
# Wrap the two linear system solves into a Linear Solver
class LinearSolver:
    
    # We allow direct call methods in the class
    @staticmethod
    def gauss_jacobi(A, b, tol=1e-10):
        x = np.zeros_like(b)
        N = np.diagflat(np.diag(A))
        P = A - N
        
        while True:
            pre_x = x.copy()
            x = np.linalg.solve(N, b - P.dot(pre_x))
            if np.sum((x - pre_x) ** 2) < tol:
                print('Successfully found solutions with Jacobi method:', x)
                break
        
        print("Results for Ax - b =", A.dot(x) - b)
        return x
    
    @staticmethod
    def gauss_seidel(A, b, w=1, shuffle=False, tol=1e-10):
        x = np.zeros_like(b)
        
        if shuffle:
            indices = np.arange(len(b))
            np.random.shuffle(indices)
            A = A[indices]
            b = b[indices]
        
        L = np.tril(A)
        U = A - L
        
        while True:
            pre_x = x.copy()
            x = w * np.linalg.solve(L, b - U.dot(pre_x)) + (1 - w) * pre_x
            if np.sum((x - pre_x) ** 2) < tol:
                print('Successfully found solutions with Gauss-Seidel method:', x)
                break
        
        print("Results for Ax - b =", A.dot(x) - b)
        return x


In [13]:
A = np.array([[1,.5,.3],[.6,1,.1],[.2,.4,1]])
b = np.array([5.,7.,4.])


In [14]:
lin_solver = LinearSolver()
lin_solver.gauss_jacobi(A,b)
lin_solver.gauss_seidel(A,b)

Successfully found solutions with Jacobi method: [1.67155639 5.86510467 1.31964982]
Results for Ax - b = [3.67428675e-06 3.48811842e-06 2.97013887e-06]
Successfully found solutions with Gauss-Seidel method: [1.67155553 5.86510185 1.31964815]
Results for Ax - b = [ 9.00075156e-07 -1.41752707e-08  0.00000000e+00]


array([1.67155553, 5.86510185, 1.31964815])